The following notebook will be split into 4 parts:
 - [Part 1](#Part1_REFIT): Setup
 - [Part 2](#Part2_REFIT): Exploratory Data Analysis
 - [Part 3](#Part3_REFIT): Clustering
 - [Part 4](#Part4_REFIT): Forecasting
 
**Part 1:** Initial setup (importing relevant packages, setting up global hyperparameters, importing/cleaning our data set).

**Part 2:** Exploratory data analysis that serves to act as the feature selection step by determining which features are ir/relevant.

**Part 3:** Clustering days based on a similarity metric.

**Part 4:** Forecasting on a per-cluster basis.

# Part 1: Setup <a id="Part1_REFIT"> </a>

## 1.1: Initial setup

**Step 1:** Import the relevant packages and set Seaborn/Matplotlib hyperparameters.

In [ ]:
import hdbscan
import holidays
import os
import sklearn
import tensorflow
import umap

import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from scipy import stats
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, ConvLSTM2D, Dense, Flatten, MaxPooling1D, RepeatVector, TimeDistributed, MaxPooling2D, LeakyReLU
from tensorflow.keras.models import Sequential, load_model
from tqdm import notebook

plt.style.use("fivethirtyeight")
sns.set(style="whitegrid", palette="muted")

plt.rcParams["axes.labelsize"] = 26
plt.rcParams["axes.titlesize"] = 26
plt.rcParams["figure.figsize"] = 16, 10
plt.rcParams["figure.dpi"] = 300
plt.rcParams["xtick.labelsize"] = 22
plt.rcParams["ytick.labelsize"] = 22
plt.rcParams["legend.fontsize"] = 22

np.set_printoptions(suppress=True)
pd.options.display.float_format = "{:.2f}".format

tensorflow.random.set_seed(3141589)

%config Completer.use_jedi = False

**Step 2:** Define the location of our data as well as the relevant columns that we would like to include.

In [ ]:
house_number = 12

cols_REFIT = [
    "Time",
    "Aggregate",
    "Appliance1",
    "Appliance2",
    "Appliance3",
    "Appliance4",
    "Appliance5",
    "Appliance6",
    "Appliance7",
    "Appliance8",
    "Appliance9",
]

data_directory_REFIT = os.path.join("Data", "REFIT")
data_directory_Solcast = os.path.join("Data", "Solcast_REFIT")

house = f"CLEAN_House{house_number}.csv"
solcast_15 = "Solcast_REFIT_15.csv"

file_destination_REFIT = os.path.join(data_directory_REFIT, house)
file_destination_Solcast = os.path.join(data_directory_Solcast, solcast_15)

**Step 3:** Read in the data and save it to a dataframe.

In [ ]:
df_REFIT = pd.read_csv(file_destination_REFIT, index_col=0, parse_dates=True, usecols=cols_REFIT)
df_Solcast = pd.read_csv(file_destination_Solcast, index_col=0, parse_dates=True)

df_Solcast.index = df_Solcast.index.rename("Time")
df_Solcast.index = pd.to_datetime(df_Solcast.index).tz_localize(None)

cols_REFIT.remove("Time")

## 1.2: Scale the data in the dataframe(s)

**Step 1.1:** Scale the data in a range between 0 and 1 (optional).

In [ ]:
# minmax_REFIT = MinMaxScaler()
# minmax_Solcast = MinMaxScaler()

# df_REFIT[cols_REFIT] = minmax_REFIT.fit_transform(df_REFIT[cols_REFIT])
# df_Solcast[cols_Solcast] = minmax_Solcast.fit_transform(df_Solcast[cols_Solcast])

**Step 1.2:** Standardize the data by removing the mean and scaling to unit variance (optional).

In [ ]:
# standardscale_REFIT = StandardScaler()
# standardscale_Solcast = StandardScaler()

# df_REFIT[cols_REFIT] = standardscale_REFIT.fit_transform(df_REFIT[cols_REFIT])
# df_Solcast[cols_Solcast] = standardscale_Solcast.fit_transform(df_Solcast[cols_Solcast])

## 1.3: Merge the dataframes

**Step 1:** Create a copy of our REFIT dataframe that is resampled into a resolution of 15 minutes and drop any days that contain an incomplete number of values.

In [ ]:
df_REFIT_resampled = df_REFIT.resample("15min").mean()
df_REFIT_resampled = df_REFIT_resampled.dropna()

mask = df_REFIT_resampled.groupby(df_REFIT_resampled.index.date).size()
mask = mask[mask < 96].index.to_list()

df_REFIT_resampled = df_REFIT_resampled[~df_REFIT_resampled.index.floor("D").isin(mask)]

**Step 2:** Create a third dataframe that is the result of merging the Solcast dataframe with the REFIT dataframe.

In [ ]:
df_Merged = pd.merge(left=df_Solcast, left_on=df_Solcast.index, right=df_REFIT_resampled, right_on=df_REFIT_resampled.index)

cols_Merged = [
    "PeriodStart",
    "Period",
    "Appliance1",
    "Appliance2",
    "Appliance3",
    "Appliance4",
    "Appliance5",
    "Appliance6",
    "Appliance7",
    "Appliance8",
    "Appliance9",
]

df_Merged.drop(cols_Merged, axis=1, inplace=True)
df_Merged.rename(columns={"key_0": "Time"}, inplace=True)
df_Merged = df_Merged.set_index("Time")
df_Merged.index = pd.to_datetime(df_Merged.index)
df_Merged.head()

## 1.4: Append temporal features to our merged dataframe

**Step 1:** Append public holidays to our merged dataframe.

In [ ]:
UK_holidays = holidays.UnitedKingdom()
df_Merged.insert(0, "Holiday", [1 if str(val).split()[0] in UK_holidays else 0 for val in df_Merged.index.date])
df_Merged["Holiday"] = df_Merged["Holiday"].astype("category")

**Step 2:** Define day of the year ranges for each of the seasons.

In [ ]:
spring = range(79, 172)
summer = range(172, 266)
fall = range(266, 355)

def season(doy):
    if doy in spring:
        return "0"
    if doy in summer:
        return "1"
    if doy in fall:
        return "2"
    else:
        return "3"

**Step 3:** Append temporal data to our merged dataframe.

In [ ]:
df_Merged.insert(0, "Year", df_Merged.index.year)
df_Merged.insert(1, "Month", df_Merged.index.month)
df_Merged.insert(3, "Day", df_Merged.index.day)
df_Merged.insert(4, "Hour", df_Merged.index.hour)
df_Merged.insert(5, "Minute", df_Merged.index.minute)
df_Merged.insert(6, "Weekday", df_Merged.index.weekday)
df_Merged.insert(7, "Season", df_Merged.index.dayofyear.map(season))

## Miscellaneous functions

### 1) Augmented Dickey–Fuller test

In [ ]:
def adfuller_test(series, signif=0.05, name=""):
    r = adfuller(series, autolag="AIC")
    output = {"test_statistic": round(r[0], 4), "pvalue": round(r[1], 4), "n_lags": round(r[2], 4), "n_obs": r[3]}
    p_value = output["pvalue"]

    def adjust(val, length=6):
        return str(val).ljust(length)

    print(f'      Augmented Dickey-Fuller Test on "{name}"', "\n   ", "-" * 47)
    print(f" Null Hypothesis: Data has unit root. Non-Stationary.")
    print(f" Significance Level    = {signif}")
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key, val in r[4].items():
        print(f" Critical value {adjust(key)} = {round(val, 3)}")
    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

### 2) Augmented Dickey–Fuller test w/plot

In [ ]:
def test_stationarity(series, signif=0.05, name="", ylabel=""):
    def adjust(val, length=6):
        return str(val).ljust(length)

    rolmean = series.rolling(12).mean()
    rolstd = series.rolling(12).std()

    fig, ax = plt.subplots()

    series.plot(ax=ax, alpha=0.5)
    rolmean.plot(ax=ax, alpha=0.7)
    rolstd.plot(ax=ax, alpha=0.7)

    ax.set_xlabel("")
    ax.set_ylabel(ylabel)
    ax.set_xlim(left=0, right=len(series))
    plt.legend(loc="best")
    plt.title("Rolling Mean & Standard Deviation")
    plt.setp(ax.get_xticklabels(), ha="right", rotation=60)

    leg = plt.legend()
    leg.get_texts()[0].set_text(name)
    leg.get_texts()[1].set_text("Rolling Mean")
    leg.get_texts()[2].set_text("Rolling STD")

    plt.tight_layout()
    plt.show(block=False)

    adfuller_test(series, 0.05, name)

### 3) Granger Causality test

In [ ]:
def grangers_causation_matrix(data, variables, test="ssr_chi2test", maxlag=12, verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = ([round(test_result[i + 1][0][test][1], 2) for i in range(maxlag)])
            if verbose:
                print(f"Y = {r}, X = {c}, P Values = {p_values}")
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + "_x" for var in variables]
    df.index = [var + "_y" for var in variables]
    return df

### 4) Determine which highly correlated independent variables have a stronger correlation with our target variable.

In [ ]:
def correlation(df, threshold, target_variable):
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                rowname = corr_matrix.index[j]
                cor1 = abs(df[colname].corr(target_variable))
                cor2 = abs(df[rowname].corr(target_variable))
                if  cor1 > cor2:
                    col_corr.add(corr_matrix.index[j])
                else:
                    col_corr.add(corr_matrix.columns[i])
    return col_corr

### 5) Reshape correlation matrix

In [ ]:
def reshape_corr(df):
    df_corr = df.corr().stack().reset_index()
    df_corr.columns = ["Feature 1", "Feature 2", "Correlation"]
    mask_dups = (df_corr[["Feature 1", "Feature 2"]].apply(frozenset, axis=1).duplicated()) | (df_corr["Feature 1"] == df_corr["Feature 2"])
    df_corr = df_corr[~mask_dups]

    return df_corr

### 6) Forecasting accuracy metrics

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.round(np.mean(np.abs(forecast - actual) / np.abs(actual)) * 100, 2)
    mae = np.round(np.mean(np.abs(forecast - actual)), 2)
    rmse = np.round(np.mean((forecast - actual) ** 2) ** 0.5, 2)

    print("Forecasting accuracy metrics:")
    print(f"\t - MAPE = {mape}%")
    print(f"\t - MAE = {mae}")
    print(f"\t - RMSE = {rmse}")

### 7) Reshape data into a suitable format for single step forecasting using our CNN-LSTM architecture

In [ ]:
def to_Supervised(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i : (i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

### 8) Reshape data into a suitable format for multi-step forecasting using our CNN-LSTM architecture

In [ ]:
def to_Supervised_ms(ts: np.array, lag=1, n_ahead=1, target_index=0) -> tuple:
    n_features = ts.shape[1]
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag - n_ahead):
            Y.append(ts[(i + lag) : (i + lag + n_ahead), target_index])
            X.append(ts[i : (i + lag)])
    X, Y = np.array(X), np.array(Y)

    X = np.reshape(X, (X.shape[0], lag, n_features))

    return X, Y

# Part 2: Exploratory Data Analysis <a id="Part2_REFIT"> </a>

## 2.1: Enumerate values of interest

**Step 1:** We iterate over each of the houses available in our data set and calculate the total number of missing days, total number of days that contain *incomplete* data, total number of values recorded as well as the number of values that contain *issues* and save the results to a `.csv` file.

In [ ]:
output = {
    "House number": [],
    "Date start": [],
    "Date end": [],
    "Total no. of days": [],
    "No. of days missing": [],
    "% of days missing": [],
    "No. of days incomplete": [],
    "% of days incomplete": [],
    "Total no. of values recorded": [],
    "No. of values missing data": [],
    "% of values missing data": [],
    "Number of values with issues": [],
    "% of values with issues": [],
    "Longest period of missing days": [],
}

for i in notebook.trange(1, 22):
    if i == 14:
        continue

    house_number = i
    house = f"CLEAN_House{house_number}.csv"
    file_destination = os.path.join(data_directory_REFIT, house)
    df = pd.read_csv(file_destination, index_col=0, parse_dates=True)

    # Define the starting date and ending date of our dataframe.
    date_start = df.index.min()
    date_end = df.index.max()

    df_grouped = df.groupby(df.index.date).mean()
    total_values = len(df)
    total_days = len(pd.date_range(df_grouped.index.min(), df_grouped.index.max()))
    
    # Set up a dataframe with which to calculate incomplete days.
    df_r = df.resample("15min").mean()
    df_r = df_r.dropna()

    # Count the number of missing days.
    days_missing = len(pd.date_range(df_grouped.index.min(), df_grouped.index.max()).difference(df_grouped.index))
    percentage_days_missing = np.round((days_missing / total_days) * 100, 2)

    # Count the number of days with incomplete data.
    mask = df_r.groupby(df_r.index.date).size()
    mask = mask[mask < 96].index.to_list()
    days_incomplete = len(mask)
    percentage_days_incomplete = np.round((days_incomplete / total_days) * 100, 2)

    # Count the number of missing values.
    values_missing = (df["Aggregate"] == 0).astype(int).sum()
    percentage_values_missing = np.round((values_missing / total_values) * 100, 2)

    # Count the number of values with 'issues'.
    values_issues = (df["Issues"] == 1).astype(int).sum()
    percentage_values_issues = np.round((values_issues / total_values) * 100, 2)

    # Calculate the longest period of missing days.
    df_grouped["Diff_Date"] = df_grouped.index.to_series().diff()
    df_grouped["Diff_Date"] = df_grouped["Diff_Date"] / np.timedelta64(1, "D")
    stretch = df_grouped["Diff_Date"].max().astype(int) - 1

    output["House number"].append(house_number)
    output["Date start"].append(date_start)
    output["Date end"].append(date_end)
    output["Total no. of days"].append(total_days)
    output["No. of days missing"].append(days_missing)
    output["% of days missing"].append(percentage_days_missing)
    output["Total no. of values recorded"].append(total_values)
    output["No. of days incomplete"].append(days_incomplete)
    output["% of days incomplete"].append(percentage_days_incomplete)
    output["No. of values missing data"].append(values_missing)
    output["% of values missing data"].append(percentage_values_missing)
    output["Number of values with issues"].append(values_issues)
    output["% of values with issues"].append(percentage_values_issues)
    output["Longest period of missing days"].append(stretch)
    
output = pd.DataFrame.from_dict(output).set_index("House number")
output.to_csv("Data Analysis - REFIT.csv")

## 2.2: Visualize the data

### 2.2.1: Line plot

**Step 1:** Create a copy of our REFIT dataframe and reformat the index into a string so as to make sure that Matplotlib does not automatically interpolate missing values.

In [ ]:
df_REFIT_resampled_c = df_REFIT_resampled.copy()
df_REFIT_resampled_c.index = df_REFIT_resampled_c.index.strftime("%d-%m-%y %H:%M:%S")

**Step 2:** Drop all columns barre the `Aggregate` column.

In [ ]:
cols_REFIT_c = cols_REFIT.copy()
cols_REFIT_c.remove("Aggregate")
df_REFIT_resampled_c.drop(cols_REFIT_c, axis=1, inplace=True)

**Step 3:** Plot the aggregate power consumption over time.

In [ ]:
fig, ax = plt.subplots()
df_REFIT_resampled_c["Aggregate"].plot(ax=ax)

ax.set_xlabel("")
ax.set_ylabel("Aggregate Power Consumption (Watts)")
ax.set_xlim(left=-5, right=len(df_REFIT_resampled_c) + 5)
ax.set_ylim(bottom=df_REFIT_resampled_c["Aggregate"].min() - 100)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

### 2.2.2: Stack plot

**Step 1:** Create a copy of our REFIT dataframe that contains only the IAM readings (with their labels) and group by the averaged reading per hour of the day.

In [ ]:
hours = []
for i in range(0, 24):
    if i < 10:
        hours.append(f"0{i}:00:00")
    if i >= 10:
        hours.append(f"{i}:00:00")

In [ ]:
df_Stacked = df_REFIT.iloc[:, 1:10].groupby(df_REFIT.index.hour).mean()

df_Stacked = df_Stacked.rename(
    index=str,
    columns={
        "Appliance1": "Fridge-Freezer",
        "Appliance2": "Unknown",
        "Appliance3": "Unknown",
        "Appliance4": "Computer Site",
        "Appliance5": "Microwave",
        "Appliance6": "Kettle",
        "Appliance7": "Toaster",
        "Appliance8": "Television",
        "Appliance9": "Unknown",
    },
)
df_Stacked.index = hours
df_Stacked.index = pd.to_datetime(df_Stacked.index, format="%H:%M:%S")
df_Stacked

**Step 2:** Create the stack plot.

In [ ]:
fig, ax = plt.subplots()

colors = ["b", "orange", "g", "r", "purple", "brown", "pink", "gray", "yellow"]

ax.stackplot(df_Stacked.index.values, np.transpose(df_Stacked.values), labels=df_Stacked.columns.values, colors=colors)
ax.legend(loc="upper left", fontsize=18)
ax.set_xlim(df_Stacked.index.min(), df_Stacked.index.max())
ax.xaxis.set_major_locator(md.HourLocator())
ax.xaxis.set_major_formatter(md.DateFormatter("%H:%M"))

plt.xlabel("Time")
plt.ylabel("Average Power Consumption (Watts)")
plt.xticks(df_Stacked.index.values)

fig.autofmt_xdate()

plt.tight_layout()
plt.show()

**Step 3:** Create individual plots for each of the IAMs.

In [ ]:
fig, axs = plt.subplots(3, 3)
i = 0
colors = ["b", "orange", "g", "r", "purple", "brown", "pink", "gray", "yellow"]

list = []
for x in range(0, 24, 6):
    list.append(df_Stacked.index[x])
list.append(df_Stacked.index[-1])

for nrow in range(0, 3):
    for ncol in range(0, 3):
        axs[nrow, ncol].plot(df_Stacked.index, df_Stacked.iloc[:, i], color=colors[i])
        axs[nrow, ncol].set_xlim(df_Stacked.index.min(), df_Stacked.index.max())
        axs[nrow, ncol].set_title(df_Stacked.columns[i], fontsize=18)
        axs[nrow, ncol].xaxis.set_major_locator(md.HourLocator())
        axs[nrow, ncol].xaxis.set_major_formatter(md.DateFormatter("%H:%M"))
        axs[nrow, ncol].set_xticks(list)
        i = i + 1

fig.autofmt_xdate()
fig.add_subplot(111, frameon=False)

plt.grid(False)
plt.tick_params(labelcolor="none", top=False, bottom=False, left=False, right=False)
plt.xlabel("Time", labelpad=25)
plt.ylabel("Average Power Consumption (Watts)", labelpad=15)
plt.tight_layout()
plt.show()

### 2.2.3: Count plot

**Step 1:** Visualize the number of samples per day of the week over the entire data set.

In [ ]:
df_Merged_c = df_Merged.copy()
df_Merged_c.insert(0, "Weekday_name", df_Merged.index.day_name())

ax = sns.countplot(
    x=df_Merged_c["Weekday_name"],
    data=df_Merged_c,
    palette="icefire",
    order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
)

plt.xlabel("Weekday")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

**Step 2:** Visualize the number of samples per month over the entire data set.

In [ ]:
df_Merged_c = df_Merged.copy()
df_Merged_c.insert(0, "Month_name", df_Merged.index.month_name())

ax = sns.countplot(
    x=df_Merged_c["Month_name"],
    data=df_Merged_c,
    palette="icefire",
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

### 2.2.4: Stationarity plot

**Step 1:** Test for stationarity and plot the result using the pre-defined `test_stationarity` function.

In [ ]:
test_stationarity(df_REFIT_resampled_c, 0.05, "Aggregate", "Aggregate Power Consumption (Watts)")

### 2.2.5: Box and whisker plots/outliers

**Step 1:** Box and whiskers plot for the aggregate as well as each of the IAM readings grouped by the months of the year.

In [ ]:
fig, axs = plt.subplots(len(cols_REFIT), 1, figsize=(16, 10 * (len(cols_REFIT))), sharex=True)

for col, ax in zip([*cols_REFIT], axs):
    sns.boxplot(
        data=df_REFIT,
        x=df_REFIT.index.month_name(),
        y=col,
        ax=ax,
        order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    )
    ax.set_title(col)
    ax.xaxis.set_label_text("")

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.tight_layout()
plt.show()

**Step 2:** We remove outliers that are 3 standard deviations away from the mean and repeat **step 1**.

In [ ]:
df_out = df_REFIT[(np.abs(stats.zscore(df_REFIT["Aggregate"])) < 3)]
print(f"Number of outliers: {(len(df_REFIT) - len(df_out))}")

In [ ]:
fig, axs = plt.subplots(len(cols_REFIT), 1, figsize=(16, 10 * (len(cols_REFIT))), sharex=True)

for col, ax in zip([*cols_REFIT], axs):
    sns.boxplot(
        data=df_out,
        x=df_out.index.month_name(),
        y=col,
        ax=ax,
        order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    )
    ax.xaxis.set_label_text("")

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.tight_layout()
plt.show()

**Step 2.1:** Purely for the aggregate power consumption.

In [ ]:
ax = sns.boxplot(
    data=df_out,
    x=df_out.index.month_name(),
    y=df_out["Aggregate"],
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Aggregate Power Consumption (Watts)")
plt.tight_layout()
plt.show()

**Step 3:** Box and whiskers plot for the aggregate as well as each of the IAM readings over the entirety of the data set.

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_out, ax=ax)

plt.setp(ax.get_xticklabels(), ha="right", rotation=45)
plt.tight_layout()
plt.show()

## 2.3: Time series decomposition

**Step 1:** Create a copy of our REFIT dataframe and reformat the index into a string so as to make sure that Matplotlib does not automatically interpolate missing values.

In [ ]:
df_REFIT_resampled_c = df_REFIT_resampled.copy()
df_REFIT_resampled_c.index = df_REFIT_resampled_c.index.strftime("%d-%m-%y %H:%M:%S")

**Step 2:** Drop all columns barre the `Aggregate` column.

In [ ]:
cols_REFIT_c = cols_REFIT.copy()
cols_REFIT_c.remove("Aggregate")
df_REFIT_resampled_c.drop(cols_REFIT_c, axis=1, inplace=True)

**Step 3:** Define a period of 2-3 months (as an example).

In [ ]:
df_REFIT_resampled_c = df_REFIT_resampled_c.loc["2015-01-01":"2015-04-01"]

**Step 4:** Perform time series decomposition using LOESS.

In [ ]:
freq = (24 * 60) // 15
stl_decompose_result = STL(df_REFIT_resampled_c, period=freq).fit()

**Step 5:** Separate/plot the obtained results.

In [ ]:
observed = stl_decompose_result.observed
trend = stl_decompose_result.trend.to_frame()
seasonal = stl_decompose_result.seasonal.to_frame()
noise = stl_decompose_result.resid.to_frame()

In [ ]:
fig, axs = plt.subplots(4, sharex=True)

observed.plot(ax=axs[0], title="Observed")
trend.plot(ax=axs[1], title="Trend")
seasonal.plot(ax=axs[2], title="Seasonal")
noise.plot(ax=axs[3], title="Noise")

for ax in axs:
    ax.set_xlabel("")
    ax.set_xlim(0, right=len(observed))
    ax.get_legend().remove()
    ax.title.set_size(22)

fig.text(0.03, 0.6, "Aggregate Power Consumption (Watts)", fontsize="22", va="center", rotation="vertical")
plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

## 2.4: Causality and correlation

**Step 1:** Perform the Augmented Dicky-Fuller test to determine whether our time series is stationary.

In [ ]:
for name, column in df_Merged.iteritems():
    adfuller_test(column, name=column.name)
    print("\n")

**Step 2:** Check for autocorrelation.

In [ ]:
plot_acf(df_REFIT_resampled["Aggregate"], lags=50, zero=False)
plt.show()

**Step 3:** Check for partial autocorrelation.

In [ ]:
plot_pacf(df_REFIT_resampled["Aggregate"], lags=50, zero=False)
plt.show()

**Step 4:** Check for highly correlated features in the Solcast dataframe and drop them from our merged dataframe.

In [ ]:
highly_correlated = correlation(df_Solcast, 0.7, df_Merged.Aggregate)
A = len(df_Merged.columns)
print(highly_correlated)
#df_Merged.drop(highly_correlated, axis=1, inplace=True)
B = len(df_Merged.columns) - len(highly_correlated)
print(A, B)

**Step 5:** Estimate mutual information of our independent variables on our target variable.

In [ ]:
X, y = (df_Merged.loc[:, df_Merged.columns != "Aggregate"], df_Merged["Aggregate"])
mutual_info = mutual_info_regression(X, y)

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info = mutual_info.sort_values(ascending=False)

In [ ]:
ax = sns.barplot(x=mutual_info.values, y=mutual_info.index, palette="icefire")
ax.set_xlabel("Mutual information")
ax.set_ylabel("Independent variable")
plt.tight_layout()
plt.show()

**Step 6:** Granger causality test.

In [ ]:
gm = grangers_causation_matrix(df_Merged, variables = df_Merged.columns) 

In [ ]:
gm2 = gm.transpose()["Aggregate_y"].iloc[0:18].to_frame()
ax = sns.heatmap(gm2, mask=gm2 < 0.05, annot=True, fmt=".2f", square=True, cmap="rocket_r", vmin=0.0, vmax=1.0)
ax2 = sns.heatmap(gm2, mask=gm2 >= 0.05, annot=True, fmt=".2f", square=True, cmap="rocket_r", vmin=0.0, vmax=1.0, annot_kws={"weight": "bold"}, cbar=False)

plt.show()

In [ ]:
ax = sns.heatmap(gm.transpose(), annot=True, fmt=".2f", square=True, cmap="rocket_r")

# Part 3: Clustering <a id="Part3_REFIT"> </a>

## 3.1: Dimensionality reduction

**Step 1:** Create a copy of our REFIT dataframe

In [ ]:
df_REFIT_resampled_c = df_REFIT_resampled.copy()

**Step 2:** Drop all columns barre the `Aggregate` column.

In [ ]:
cols_REFIT_c = cols_REFIT.copy()
cols_REFIT_c.remove("Aggregate")
df_REFIT_resampled_c.drop(cols_REFIT_c, axis=1, inplace=True)

**Step 3:** Reshape our dataframe as 96 columns that represent the 96 15-minute chunks of each day.

In [ ]:
df_REFIT_resampled_c.index = pd.MultiIndex.from_arrays([df_REFIT_resampled_c.index.date, df_REFIT_resampled_c.index.time], names=["Date", "Time"])
df_REFIT_resampled_c = df_REFIT_resampled_c.unstack()

### 3.1.1: Statistical parameters

**Step 1:** Split our day into 5 periods:
 - `LEEM`: Late evening/early morning (23:30-06:00)
 - `MR`: Morning (06:00-11:00)
 - `LMAF`: Late morning/afternoon (11:00-15:00)
 - `LAEE`: Late afternoon/early evening (15:00-20:30)
 - `EV`: Evening (20:30-23:30)

In [ ]:
LEEM = df_REFIT_resampled_c.iloc[:, np.r_[0:24, 92:96]]
MR = df_REFIT_resampled_c.iloc[:, 24:45]
LMAF = df_REFIT_resampled_c.iloc[:, 44:61]
LAEE = df_REFIT_resampled_c.iloc[:, 60:83]
EV = df_REFIT_resampled_c.iloc[:, 82:95]

**Step 2:** Create a new dataframe that consists of the mean, min, max and standard deviation of each of our 5 periods per day. We now represent each day with 20 variables rather than 96.

In [ ]:
df_SP = LEEM.mean(axis=1).to_frame(name="LEEM_Mean")
df_SP.insert(len(df_SP.columns), "LEEM_Min", LEEM.min(axis=1))
df_SP.insert(len(df_SP.columns), "LEEM_Max", LEEM.max(axis=1))
df_SP.insert(len(df_SP.columns), "LEEM_STD", LEEM.std(axis=1))
df_SP.insert(len(df_SP.columns), "MR_Mean", MR.mean(axis=1))
df_SP.insert(len(df_SP.columns), "MR_Min", MR.min(axis=1))
df_SP.insert(len(df_SP.columns), "MR_Max", MR.max(axis=1))
df_SP.insert(len(df_SP.columns), "MR_STD", MR.std(axis=1))
df_SP.insert(len(df_SP.columns), "LMAF_Mean", LMAF.mean(axis=1))
df_SP.insert(len(df_SP.columns), "LMAF_Min", LMAF.min(axis=1))
df_SP.insert(len(df_SP.columns), "LMAF_Max", LMAF.max(axis=1))
df_SP.insert(len(df_SP.columns), "LMAF_STD", LMAF.std(axis=1))
df_SP.insert(len(df_SP.columns), "LAEE_Mean", LAEE.mean(axis=1))
df_SP.insert(len(df_SP.columns), "LAEE_Min", LAEE.min(axis=1))
df_SP.insert(len(df_SP.columns), "LAEE_Max", LAEE.max(axis=1))
df_SP.insert(len(df_SP.columns), "LAEE_STD", LAEE.std(axis=1))
df_SP.insert(len(df_SP.columns), "EV_Mean", EV.mean(axis=1))
df_SP.insert(len(df_SP.columns), "EV_Min", EV.min(axis=1))
df_SP.insert(len(df_SP.columns), "EV_Max", EV.max(axis=1))
df_SP.insert(len(df_SP.columns), "EV_STD", EV.std(axis=1))

### 3.1.2: PCA

**Step 1:** Determine the cumulative explained variance ratio as a function of the number of variables.

In [ ]:
pca = PCA().fit(df_SP)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of variables")
plt.ylabel("Cumulative explained variance")
plt.tight_layout()
plt.show()

**Step 2:** Apply PCA to our data set.

In [ ]:
pca = PCA(random_state=3141589, n_components = 8)
df_SP1 = pca.fit_transform(df_SP)

### 3.1.3: UMAP

**Step 1:** Apply UMAP to our data set.

In [ ]:
projection = umap.UMAP(random_state=3141589, n_neighbors=np.power(len(df_REFIT_resampled_c), 0.5).astype(int), min_dist=0.1, n_components=2).fit_transform(df_SP)
projection = TSNE(random_state=3141589).fit_transform(projection)
plt.scatter(*projection.T)
plt.tight_layout()
plt.show()

## 3.2: HDBSCAN

**Step 1:** Define our HDBSCAN clusterer with the appropriate hyperparameters and fit it to our 2-dimensional projection.

In [ ]:
HDB = hdbscan.HDBSCAN(min_cluster_size=(len(df_REFIT_resampled_c) // 10), min_samples=15)
HDB = HDB.fit(projection)

In [ ]:
HDB.condensed_tree_.plot(select_clusters=True)
plt.show()

**Step 2:** Plot/visualize our clusters.

In [ ]:
labels = HDB.labels_
labels = [label + 1 for label in labels]
n_clusters = len(set(labels)) - (1 if 0 in labels else 0)
n_noise = list(labels).count(0)
core_samples_mask = np.zeros_like(labels, dtype=bool)

In [ ]:
unique_labels = set(labels)
colors = [plt.cm.Set1(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == 0:
        # Black used for noise.
        col = [0, 0, 0, 1]
    class_member_mask = labels == k

    xy = projection[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], "o", markerfacecolor=tuple(col), markeredgecolor="k", markersize=14)

    xy = projection[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], "o", markerfacecolor=tuple(col), markeredgecolor="k", markersize=6)
plt.title("Estimated number of clusters: %d" % n_clusters)
plt.show()

print("Estimated number of clusters: %d" % n_clusters)
print("Estimated number of noise points: %d" % n_noise)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(projection, labels))

**Step 3:** Plot/visualize the average pattern per cluster

In [ ]:
df_REFIT_resampled_c["Labels"] = labels
Cx = []

for i in range(1, n_clusters + 1):
    Cx.append(f"C" + str(i))
    globals()["C" + str(i)] = (df_REFIT_resampled_c.loc[df_REFIT_resampled_c["Labels"] == i]).mean(axis=0)
    globals()["C" + str(i)] = globals()["C" + str(i)].reset_index()
    globals()["C" + str(i)].drop("level_0", axis=1, inplace=True)
    globals()["C" + str(i)].drop(globals()["C" + str(i)].tail(1).index, inplace=True)
    globals()["C" + str(i)]["Time"] = globals()["C" + str(i)]["Time"].astype("str")
    globals()["C" + str(i)]["Time"] = pd.to_datetime(globals()["C" + str(i)]["Time"])
    globals()["C" + str(i)] = globals()["C" + str(i)].set_index("Time")
    globals()["C" + str(i)].index = globals()["C" + str(i)].index.strftime("%H:%M:%S")
    globals()["C" + str(i)] = globals()["C" + str(i)].rename(columns={0: "Aggregate"})

fig, ax = plt.subplots()
for i in range(1, n_clusters + 1):
    globals()["C" + str(i)].plot(ax=ax, color=colors[i - 1])

ax.set_xlabel("Time")
ax.set_ylabel("Aggregate Power Consumption (Watts)")
ax.set_xlim(left=0, right=95)
plt.legend([*Cx], loc="best", fontsize=18)
plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

In [ ]:
df_REFIT_resampled_c.insert(0, "Month_name", df_REFIT_resampled_c.index.month_name())

In [ ]:
ax = sns.countplot(
    x="Month_name",
    hue="Labels",
    data=df_REFIT_resampled_c,
    palette=colors,
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
df_REFIT_resampled_c.insert(0, "Day_name", df_REFIT_resampled_c.index.day_name())

In [ ]:
ax = sns.countplot(
    x="Day_name",
    hue="Labels",
    data=df_REFIT_resampled_c,
    palette=colors,
    order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Day")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# Part 4: Forecasting <a id="Part4_REFIT"> </a>

In [ ]:
C1_Merge = df_REFIT_resampled_c.loc[df_REFIT_resampled_c["Labels"] == 1].copy()
C1_Merge = C1_Merge.stack().reset_index()
C1_Merge["Date"] = C1_Merge["Date"].astype("str")
C1_Merge["Time"] = C1_Merge["Time"].astype("str")
C1_Merge["DT"] = C1_Merge["Date"].str.cat(C1_Merge["Time"], sep=" ")

cols_NA = [
    "Date",
    "Time",
    "Labels",
]

C1_Merge.drop(cols_NA, axis=1, inplace=True)
C1_Merge["DT"] = pd.to_datetime(C1_Merge["DT"])
C1_Merge = C1_Merge.set_index("DT")
Test = df_Merged.copy()
Test = Test.loc[C1_Merge.index]
Test["Holiday"] = Test["Holiday"].astype("int32")
Test["Season"] = Test["Season"].astype("int32")

In [ ]:
freq = (24 * 60) // 15
stl_decompose_result = STL(Test.Aggregate, period=freq).fit()
Test.insert(len(Test.columns), "Trend", stl_decompose_result.trend.values)

In [ ]:
scaler = MinMaxScaler()
Test[Test.columns] = scaler.fit_transform(Test[Test.columns])
scaler.min_, scaler.scale_ = scaler.min_[len(Test.columns) - 1], scaler.scale_[len(Test.columns) - 1]

In [ ]:
time_steps = 24
Split = len(Test)
s_Train = Test.iloc[0 : int(Split * 0.6)]
s_Val = Test.iloc[int(Split * 0.6): int(Split * 0.8)]
s_Test = Test.iloc[int(Split * 0.8) :]

X_train, Y_train = to_Supervised(s_Train, s_Train.Trend, time_steps)
X_val, Y_val = to_Supervised(s_Val, s_Val.Trend, time_steps)
X_test, Y_test = to_Supervised(s_Test, s_Test.Trend, time_steps)

X_train = X_train.reshape(-1, 24, 14, 2, 1)
X_val = X_val.reshape(-1, 24, 14, 2, 1)
X_test = X_test.reshape(-1, 24, 14, 2, 1)

In [ ]:
model = Sequential(
    [
        ConvLSTM2D(filters=32, kernel_size=(3, 1), activation="relu", padding="same", input_shape=(time_steps, X_train.shape[2], X_train.shape[3], 1)),
        MaxPooling2D(2),
        LeakyReLU(),
        Flatten(),
        RepeatVector(1),
        LSTM(128, activation="relu", return_sequences=True),
        LSTM(64, activation="relu", return_sequences=True),
        TimeDistributed(Dense(128, activation="relu")),
        TimeDistributed(Dense(1)),
    ]
)

model.summary()
model.compile(optimizer="Adam", loss="mse")

In [ ]:
reduce_learning = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2, verbose=1, mode="auto", min_delta=0.0001, cooldown=4, min_lr=0)
eary_stopping = EarlyStopping(monitor="val_loss", min_delta=0, patience=7, verbose=1, mode="auto")
callbacks = [reduce_learning, eary_stopping]

In [ ]:
history = model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=24,
    validation_data=(X_val, Y_val),
    verbose=1,
    shuffle=False,
    callbacks=callbacks,
)

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs_graph = np.arange(1, len(loss) + 1)

x_ticks = np.arange(0, len(loss) + 1, 5)
x_ticks = np.insert(x_ticks, 1, 1)
x_ticks = np.insert(x_ticks, len(x_ticks), 39)

plt.xticks(x_ticks)
plt.xlim(1, 39)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(epochs_graph, loss, "red", label="Training loss")
plt.plot(epochs_graph, val_loss, "blue", label="Validation loss")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# model.save("Models/REFIT_Model")

In [ ]:
model = load_model("Models/REFIT_Model")
model.summary()

In [ ]:
Predictions = model.predict(X_test)
Predictions = scaler.inverse_transform(Predictions.reshape(-1, 1))

In [ ]:
s_Pred = s_Test["Trend"].to_frame().copy()
s_Pred = s_Pred.iloc[time_steps:]
s_Pred[s_Pred.columns] = scaler.inverse_transform(s_Pred[s_Pred.columns])
s_Pred.insert(len(s_Pred.columns), "Predictions", Predictions)
s_Pred.index = s_Pred.index.strftime("%d-%m-%y")

In [ ]:
fig, ax = plt.subplots()
s_Pred.Predictions.plot(ax=ax, label="Predictions")
s_Pred.Trend.plot(ax=ax, label="Ground truth")
ax.set_xlabel("Time")
ax.set_ylabel("Aggregate Power Consumption (Watts)")
ax.set_xlim(0, len(s_Pred))
fig.autofmt_xdate()
plt.legend()
plt.tight_layout()
plt.show()

forecast_accuracy(s_Pred.Predictions.ravel(), s_Pred.Trend.ravel())

In [ ]:
X_train, Y_train = to_Supervised_ms(s_Train.values, lag=24, n_ahead=12, target_index=len(s_Train.columns) - 1)
X_val, Y_val = to_Supervised_ms(s_Val.values, lag=24, n_ahead=12, target_index=len(s_Val.columns) - 1)
X_test, Y_test = to_Supervised_ms(s_Test.values, lag=24, n_ahead=12, target_index=len(s_Test.columns) - 1)

X_train = X_train.reshape(-1, 24, 14, 2, 1)
X_val = X_val.reshape(-1, 24, 14, 2, 1)
X_test = X_test.reshape(-1, 24, 14, 2, 1)

In [ ]:
model = Sequential(
    [
        ConvLSTM2D(filters=32, kernel_size=(3, 1), activation="relu", padding="same", input_shape=(time_steps, X_train.shape[2], X_train.shape[3], 1)),
        MaxPooling2D(2),
        LeakyReLU(),
        Flatten(),
        RepeatVector(1),
        LSTM(128, activation="relu", return_sequences=True),
        LSTM(64, activation="relu", return_sequences=True),
        TimeDistributed(Dense(128, activation="relu")),
        TimeDistributed(Dense(12)),
    ]
)

model.summary()
model.compile(optimizer="Adam", loss="mse")

In [ ]:
history = model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=48,
    validation_data=(X_val, Y_val),
    verbose=1,
    shuffle=False,
    callbacks=callbacks,
)

In [ ]:
# model.save("Models/REFIT_Model_12_Steps_Ahead")

In [ ]:
model = load_model("Models/REFIT_Model_12_Steps_Ahead")
model.summary()

In [ ]:
Predictions = model.predict(X_test)

In [ ]:
Predictions = scaler.inverse_transform(Predictions.reshape(-1, 1))
Y_test = scaler.inverse_transform(Y_test.reshape(-1, 1))

In [ ]:
forecast_accuracy(Predictions.ravel(), Y_test.ravel())